In [1]:
import warnings
import time
import sys
import os
import datetime
from sklearn.metrics import mean_squared_error
from sklearn.metrics import log_loss
from model_zoo import my_lgb
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn import linear_model
import pickle as pkl
%matplotlib inline

In [2]:
# 评分函数与提交函数
def score(pre, truth):
    return 1 / (MAE(pre, truth) + 1)

def MAE(pre, truth):
    return abs((np.rint(pre) - truth)).mean()

def submit(model_name='default', predictions=None):
    sub_df = pd.read_csv('dataset/submit_example.csv')
    sub_df[' score'] = np.rint(predictions).astype(int)
    sub_df.sort_values('score', inplace=True)
    
    sub_df.to_csv("output/{}.csv".format(model_name), index=False)

In [3]:
train_df = pd.read_csv('dataset/train_dataset.csv')
test_df = pd.read_csv('dataset/test_dataset.csv')

特征工程，简单粗暴。

* 长尾数据处理

* 过拟合

In [12]:
def percent(df, feature):
    for f in feature:
        data[f] = np.rint(data[f])
        new_feature = f + '_count'
        tmp = df.groupby(f).size().reset_index().rename(columns={0:new_feature})
        #  tmp[new_feature] = tmp[new_feature].cumsum()*100/df.shape[0]
        df = df.merge(tmp, 'left', on = f)
    return df
    
def generate_feature(df):
    df['用户前五个月平均消费值（元）'] = (df['用户近6个月平均消费值（元）']*6 - df['用户账单当月总费用（元）'])/5
    df['当月消费值较前五个月平均消费值'] = df['用户账单当月总费用（元）'] - df['用户前五个月平均消费值（元）']
    
    app_col = ['当月视频播放类应用使用次数','当月金融理财类应用使用总次数','当月网购类应用使用次数']
    # df['是否使用网购类应用'] = np.where(df['当月网购类应用使用次数'] > 0, 1, 0)
    df['当月网购类应用使用次数' + '百分比'] = (df['当月网购类应用使用次数'])/(df[app_col].sum(axis=1) + 1e-8)
    
    df.loc[df['用户年龄']==0, '用户年龄'] = df['用户年龄'].mode()
    # df.loc[df['用户话费敏感度']==0,'用户话费敏感度'] = df['用户话费敏感度'].mode()
    
    long_tail = ['当月视频播放类应用使用次数', '当月金融理财类应用使用总次数','当月网购类应用使用次数',
                 '缴费用户最近一次缴费金额（元）','用户当月账户余额（元）', '用户账单当月总费用（元）',
                 '用户近6个月平均消费值（元）','当月通话交往圈人数','当月物流快递类应用使用次数',
                '当月飞机类应用使用次数','当月火车类应用使用次数', '当月旅游资讯类应用使用次数'] 
    for feature in long_tail:
        threshold = int(df[feature].std()*3 + df[feature].mean())
        df[feature] = df[feature].apply(lambda x: x if x < threshold else threshold)
    
    df['用户最近一次缴费距今时长（月）'] = df['缴费用户最近一次缴费金额（元）'].apply(lambda x: 1 if x>0 else 0)

    return df

In [13]:
target = train_df['信用分']
data = pd.concat([train_df.drop(columns=['信用分']), test_df], axis=0, ignore_index=True)
data = generate_feature(data)
train = data.loc[:49999, :]
test = data.loc[50000:, :]

In [14]:
drop_columns = ['用户编码','是否大学生客户','用户实名制是否通过核实',
                '当月是否逛过福州仓山万达', '当月是否到过福州山姆会员店']
X_train = train.drop(columns=drop_columns).values
y_train = target.values
X_test = test.drop(columns=drop_columns).values

param = {'num_leaves': 30,
         'objective':'regression',
         'max_depth': 6,
         'learning_rate': 0.008,
         "boosting": "gbdt",
         "feature_fraction": 0.5,
         "bagging_freq": 1,
         "bagging_fraction": 0.5,
         "metric": 'mae',
         "lambda_l1": 0.15,
         "lambda_l2": 0.04,
         "verbosity": -1}

param = {'num_leaves': 35,
         'objective':'regression',
         'max_depth': 6,
         'learning_rate': 0.004,
         "boosting": "gbdt",
         "feature_fraction": 0.5,
         "bagging_freq": 1,
         "bagging_fraction": 0.5,
         "metric": 'mae',
         "lambda_l1": 0.15,
         "lambda_l2": 0.05,
         "verbosity": -1}
 94

阴差阳错我用的都是mse,只是用了不同的随机种子

In [10]:
def cache_save():
    with open('output/cache/params.pkl', 'wb') as f:
        pkl.dump(best_params, f)
    with open('output/cache/valid.pkl', 'wb') as f:
        pkl.dump(valid, f)
    with open('output/cache/res.pkl', 'wb') as f:
        pkl.dump(res, f)

In [15]:
param1 = {'num_leaves': 35,
         'objective':'regression_l2',
         'max_depth': -1,
         'learning_rate': 0.005,
         "boosting": "gbdt",
         "feature_fraction": 0.5,
         "bagging_freq": 1,
         "bagging_fraction": 0.5,
         "metric": 'mae',
         "lambda_l1": 5,
         "lambda_l2": 1,
         "verbosity": -1}
param2 = {'num_leaves': 35,
         'objective':'regression_l1',
         'max_depth': -1,
         'learning_rate': 0.005,
         "boosting": "gbdt",
         "feature_fraction": 0.5,
         "bagging_freq": 1,
         "bagging_fraction": 0.5,
         "metric": 'mae',
         "lambda_l1": 5,
         "lambda_l2": 1,
         "verbosity": -1}
valid = []
res = []
best_params = {}
for seed in [2018,2019]:
    print('fire!!!!!')
    print('*'*50)
    start = time.time()
    clf = my_lgb(folds=10, seed=seed)
    _best_params = clf.optimize_lgb(X_train, y_train, X_test, param1)
    clf.inference_folds(X_train, y_train, X_test, _best_params)
    valid.append(clf.oof)
    res.append(clf.results)
    cache_save()
    del clf
    print('*'*25)
    clf = my_lgb(folds=5, seed=seed)
    clf.inference_folds(X_train, y_train, X_test, param2)
    valid.append(clf.oof)
    res.append(clf.results)
    cache_save()
    del clf
    print('*'*25)
    print('seed:{},training spend {}s'.format(seed, time.time()-start))
    print('*'*50)

fire!!!!!
**************************************************
|   iter    |  target   | lambda_l1 | lambda_l2 | num_le... |
-------------------------------------------------------------
fold n°1
Training until validation scores don't improve for 100 rounds.
[200]	training's l1: 20.6611	valid_1's l1: 20.916
[400]	training's l1: 16.3536	valid_1's l1: 16.818
[600]	training's l1: 15.04	valid_1's l1: 15.6542
[800]	training's l1: 14.5721	valid_1's l1: 15.2818
[1000]	training's l1: 14.3467	valid_1's l1: 15.1332
[1200]	training's l1: 14.1968	valid_1's l1: 15.064
[1400]	training's l1: 14.0812	valid_1's l1: 15.0275
[1600]	training's l1: 13.9821	valid_1's l1: 15.0049
[1800]	training's l1: 13.8916	valid_1's l1: 14.9896
[2000]	training's l1: 13.8066	valid_1's l1: 14.9777
[2200]	training's l1: 13.7254	valid_1's l1: 14.9701
[2400]	training's l1: 13.6482	valid_1's l1: 14.9626
[2600]	training's l1: 13.575	valid_1's l1: 14.9565
[2800]	training's l1: 13.5028	valid_1's l1: 14.9529
[3000]	training's l1: 13.

[800]	training's l1: 14.6066	valid_1's l1: 15.0105
[1000]	training's l1: 14.3799	valid_1's l1: 14.8643
[1200]	training's l1: 14.2302	valid_1's l1: 14.7914
[1400]	training's l1: 14.1127	valid_1's l1: 14.7541
[1600]	training's l1: 14.0121	valid_1's l1: 14.7285
[1800]	training's l1: 13.9205	valid_1's l1: 14.7113
[2000]	training's l1: 13.8341	valid_1's l1: 14.699
[2200]	training's l1: 13.7523	valid_1's l1: 14.6892
[2400]	training's l1: 13.6736	valid_1's l1: 14.6831
[2600]	training's l1: 13.5981	valid_1's l1: 14.6764
[2800]	training's l1: 13.5238	valid_1's l1: 14.671
[3000]	training's l1: 13.4516	valid_1's l1: 14.666
[3200]	training's l1: 13.3818	valid_1's l1: 14.6631
[3400]	training's l1: 13.3138	valid_1's l1: 14.6593
[3600]	training's l1: 13.2479	valid_1's l1: 14.6555
[3800]	training's l1: 13.1819	valid_1's l1: 14.651
[4000]	training's l1: 13.1184	valid_1's l1: 14.6493
[4200]	training's l1: 13.0565	valid_1's l1: 14.6457
[4400]	training's l1: 12.9944	valid_1's l1: 14.6434
[4600]	training's

[1200]	training's l1: 14.2887	valid_1's l1: 14.5407
[1400]	training's l1: 14.1685	valid_1's l1: 14.4971
[1600]	training's l1: 14.065	valid_1's l1: 14.4706
[1800]	training's l1: 13.9728	valid_1's l1: 14.4548
[2000]	training's l1: 13.8838	valid_1's l1: 14.4403
[2200]	training's l1: 13.7984	valid_1's l1: 14.4316
[2400]	training's l1: 13.7191	valid_1's l1: 14.4264
[2600]	training's l1: 13.6406	valid_1's l1: 14.4222
[2800]	training's l1: 13.5651	valid_1's l1: 14.416
[3000]	training's l1: 13.4919	valid_1's l1: 14.4143
[3200]	training's l1: 13.421	valid_1's l1: 14.4109
[3400]	training's l1: 13.3504	valid_1's l1: 14.4086
[3600]	training's l1: 13.2815	valid_1's l1: 14.4071
[3800]	training's l1: 13.213	valid_1's l1: 14.4059
Early stopping, best iteration is:
[3726]	training's l1: 13.2379	valid_1's l1: 14.4048
score: 0.06391247, MAE: 14.6464
|  3        |  0.06391  |  7.086    |  2.069    |  39.65    |
fold n°1
Training until validation scores don't improve for 100 rounds.
[200]	training's l1: 20

fold n°2
Training until validation scores don't improve for 100 rounds.
[200]	training's l1: 20.7639	valid_1's l1: 21.0135
[400]	training's l1: 16.5152	valid_1's l1: 16.6466
[600]	training's l1: 15.2078	valid_1's l1: 15.3315
[800]	training's l1: 14.7418	valid_1's l1: 14.925
[1000]	training's l1: 14.5183	valid_1's l1: 14.7756
[1200]	training's l1: 14.3757	valid_1's l1: 14.7018
[1400]	training's l1: 14.2686	valid_1's l1: 14.657
[1600]	training's l1: 14.1769	valid_1's l1: 14.63
[1800]	training's l1: 14.0973	valid_1's l1: 14.611
[2000]	training's l1: 14.0203	valid_1's l1: 14.5929
[2200]	training's l1: 13.9498	valid_1's l1: 14.5834
[2400]	training's l1: 13.8814	valid_1's l1: 14.576
[2600]	training's l1: 13.8159	valid_1's l1: 14.5681
[2800]	training's l1: 13.7524	valid_1's l1: 14.5614
[3000]	training's l1: 13.6899	valid_1's l1: 14.552
[3200]	training's l1: 13.6304	valid_1's l1: 14.5477
[3400]	training's l1: 13.5732	valid_1's l1: 14.5465
Early stopping, best iteration is:
[3328]	training's l1

[800]	training's l1: 14.726	valid_1's l1: 15.141
[1000]	training's l1: 14.5009	valid_1's l1: 14.987
[1200]	training's l1: 14.3609	valid_1's l1: 14.914
[1400]	training's l1: 14.2553	valid_1's l1: 14.8691
[1600]	training's l1: 14.1675	valid_1's l1: 14.8383
[1800]	training's l1: 14.0894	valid_1's l1: 14.8155
[2000]	training's l1: 14.0154	valid_1's l1: 14.7962
[2200]	training's l1: 13.9462	valid_1's l1: 14.7812
[2400]	training's l1: 13.8804	valid_1's l1: 14.7695
[2600]	training's l1: 13.8177	valid_1's l1: 14.7576
[2800]	training's l1: 13.7577	valid_1's l1: 14.7527
[3000]	training's l1: 13.6989	valid_1's l1: 14.7439
[3200]	training's l1: 13.6428	valid_1's l1: 14.7377
[3400]	training's l1: 13.5866	valid_1's l1: 14.7318
[3600]	training's l1: 13.5325	valid_1's l1: 14.7262
[3800]	training's l1: 13.4792	valid_1's l1: 14.72
[4000]	training's l1: 13.4269	valid_1's l1: 14.7164
[4200]	training's l1: 13.375	valid_1's l1: 14.7143
Early stopping, best iteration is:
[4249]	training's l1: 13.362	valid_1'

...

fold n°3
Training until validation scores don't improve for 100 rounds.
[200]	training's l1: 20.7752	valid_1's l1: 21.0246
[400]	training's l1: 16.4867	valid_1's l1: 16.8188
[600]	training's l1: 15.1669	valid_1's l1: 15.5703
[800]	training's l1: 14.6963	valid_1's l1: 15.1672
[1000]	training's l1: 14.4738	valid_1's l1: 15.0118
[1200]	training's l1: 14.3312	valid_1's l1: 14.9444
[1400]	training's l1: 14.2245	valid_1's l1: 14.909
[1600]	training's l1: 14.1345	valid_1's l1: 14.887
[1800]	training's l1: 14.055	valid_1's l1: 14.8716
[2000]	training's l1: 13.9806	valid_1's l1: 14.8607
[2200]	training's l1: 13.9092	valid_1's l1: 14.8533
[2400]	training's l1: 13.842	valid_1's l1: 14.8473
[2600]	training's l1: 13.7787	valid_1's l1: 14.8398
[2800]	training's l1: 13.7169	valid_1's l1: 14.8375
[3000]	training's l1: 13.6551	valid_1's l1: 14.8331
[3200]	training's l1: 13.5972	valid_1's l1: 14.8296
[3400]	training's l1: 13.5402	valid_1's l1: 14.828
Early stopping, best iteration is:
[3305]	training's 

[1000]	training's l1: 14.3756	valid_1's l1: 14.8996
[1200]	training's l1: 14.2262	valid_1's l1: 14.8177
[1400]	training's l1: 14.1085	valid_1's l1: 14.7696
[1600]	training's l1: 14.0052	valid_1's l1: 14.7391
[1800]	training's l1: 13.9133	valid_1's l1: 14.7196
[2000]	training's l1: 13.8257	valid_1's l1: 14.7046
[2200]	training's l1: 13.7434	valid_1's l1: 14.6936
[2400]	training's l1: 13.6648	valid_1's l1: 14.686
[2600]	training's l1: 13.5884	valid_1's l1: 14.6776
[2800]	training's l1: 13.5141	valid_1's l1: 14.6691
[3000]	training's l1: 13.4421	valid_1's l1: 14.6677
[3200]	training's l1: 13.3731	valid_1's l1: 14.665
Early stopping, best iteration is:
[3122]	training's l1: 13.3997	valid_1's l1: 14.6642
score: 0.06391336, MAE: 14.64618
|  12       |  0.06391  |  9.984    |  2.079    |  39.98    |
fold n°1
Training until validation scores don't improve for 100 rounds.
[200]	training's l1: 20.7963	valid_1's l1: 20.8469
[400]	training's l1: 16.511	valid_1's l1: 16.6116
[600]	training's l1: 15

[2600]	training's l1: 13.633	valid_1's l1: 14.496
[2800]	training's l1: 13.5588	valid_1's l1: 14.4891
[3000]	training's l1: 13.488	valid_1's l1: 14.4882
Early stopping, best iteration is:
[2911]	training's l1: 13.5194	valid_1's l1: 14.4867
fold n°3
Training until validation scores don't improve for 100 rounds.
[200]	training's l1: 20.6592	valid_1's l1: 20.9373
[400]	training's l1: 16.3641	valid_1's l1: 16.7524
[600]	training's l1: 15.0496	valid_1's l1: 15.5328
[800]	training's l1: 14.5758	valid_1's l1: 15.1478
[1000]	training's l1: 14.3442	valid_1's l1: 14.9986
[1200]	training's l1: 14.1899	valid_1's l1: 14.9313
[1400]	training's l1: 14.0692	valid_1's l1: 14.8978
[1600]	training's l1: 13.9671	valid_1's l1: 14.8753
[1800]	training's l1: 13.8749	valid_1's l1: 14.8597
[2000]	training's l1: 13.7864	valid_1's l1: 14.849
[2200]	training's l1: 13.7035	valid_1's l1: 14.8436
[2400]	training's l1: 13.6247	valid_1's l1: 14.8393
[2600]	training's l1: 13.5499	valid_1's l1: 14.8338
[2800]	training's

[200]	training's l1: 20.8146	valid_1's l1: 20.7805
[400]	training's l1: 16.5066	valid_1's l1: 16.73
[600]	training's l1: 15.1944	valid_1's l1: 15.5137
[800]	training's l1: 14.7281	valid_1's l1: 15.1065
[1000]	training's l1: 14.5067	valid_1's l1: 14.944
[1200]	training's l1: 14.3717	valid_1's l1: 14.859
[1400]	training's l1: 14.2679	valid_1's l1: 14.806
[1600]	training's l1: 14.1776	valid_1's l1: 14.7691
[1800]	training's l1: 14.0982	valid_1's l1: 14.7492
[2000]	training's l1: 14.023	valid_1's l1: 14.7329
[2200]	training's l1: 13.9541	valid_1's l1: 14.7189
[2400]	training's l1: 13.8887	valid_1's l1: 14.7084
[2600]	training's l1: 13.8248	valid_1's l1: 14.699
[2800]	training's l1: 13.7623	valid_1's l1: 14.6902
[3000]	training's l1: 13.7026	valid_1's l1: 14.6865
[3200]	training's l1: 13.6447	valid_1's l1: 14.6795
[3400]	training's l1: 13.587	valid_1's l1: 14.6729
[3600]	training's l1: 13.5306	valid_1's l1: 14.6699
Early stopping, best iteration is:
[3643]	training's l1: 13.5183	valid_1's l

[400]	training's l1: 16.6682	valid_1's l1: 16.647
[600]	training's l1: 15.2859	valid_1's l1: 15.3776
[800]	training's l1: 14.7434	valid_1's l1: 14.9245
[1000]	training's l1: 14.4693	valid_1's l1: 14.7287
[1200]	training's l1: 14.2955	valid_1's l1: 14.6283
[1400]	training's l1: 14.1614	valid_1's l1: 14.5741
[1600]	training's l1: 14.0513	valid_1's l1: 14.54
[1800]	training's l1: 13.9517	valid_1's l1: 14.5138
[2000]	training's l1: 13.8627	valid_1's l1: 14.4974
[2200]	training's l1: 13.7791	valid_1's l1: 14.4832
[2400]	training's l1: 13.7015	valid_1's l1: 14.4741
[2600]	training's l1: 13.6267	valid_1's l1: 14.4653
[2800]	training's l1: 13.5548	valid_1's l1: 14.4587
[3000]	training's l1: 13.4869	valid_1's l1: 14.4563
[3200]	training's l1: 13.4215	valid_1's l1: 14.4518
[3400]	training's l1: 13.3585	valid_1's l1: 14.4492
[3600]	training's l1: 13.2993	valid_1's l1: 14.446
[3800]	training's l1: 13.2398	valid_1's l1: 14.4438
[4000]	training's l1: 13.1814	valid_1's l1: 14.4404
Early stopping, bes

In [23]:
val = valid[0]*0.25 + valid[1]*0.25 + valid[2]*0.25 + valid[3]*0.2511
score(val, y_train)

0.06415528658808071

In [25]:
final = res[0]*0.25 + res[1]*0.25 + res[2]*0.25 + res[3]*0.2511
submit(model_name='312', predictions=final)

In [33]:
sub_df = pd.read_csv('dataset/submit_example.csv')
sub_df[' score'] = final
# sub_df.sort_values(' score', inplace=True)
# sub_df.to_csv("output/{}.csv".format(model_name), index=False)

In [34]:
sub_df[' score'] = sub_df[' score'].apply(lambda x: int(x) if x > 650 else int(x + 1))

In [35]:
sub_df.to_csv("output/{}.csv".format('final'), index=False)

In [ ]:
param = {'num_leaves': 31,
         'objective':'regression_l2',
         'max_depth': -1,
         'learning_rate': 0.005,
         "boosting": "gbdt",
         "feature_fraction": 0.4,
         "bagging_freq": 1,
         "bagging_fraction": 0.5,
         "metric": 'mae',
         "lambda_l1": 5,
         "lambda_l2": 0,
         "verbosity":1}
clf = my_lgb(folds=5, seed=2018)
clf.inference_folds(X_train, y_train, X_test, param)
mse_2018 = clf.oof

In [ ]:
param = {'num_leaves': 31,
         'objective':'regression_l2',
         'max_depth': -1,
         'learning_rate': 0.005,
         "boosting": "gbdt",
         "feature_fraction": 0.4,
         "bagging_freq": 1,
         "bagging_fraction": 0.5,
         "metric": 'mae',
         "lambda_l1": 5,
         "lambda_l2": 0,
         "verbosity": -1}

clf_mse = my_lgb(folds=5, seed=2019)
clf_mse.inference_folds(X_train, y_train, X_test, param)
mse_2019 = clf_mse.oof

In [ ]:
p = 0.50
q = 0.502
results = mse_2018*p+ mse_2019*q 
print(score(results, y_train))
res = clf.results*p + clf_mse.results*q
submit(model_name='model_V2_bad', predictions=res)

In [ ]:
param = {'num_leaves': 35,
         'objective':'regression_l1',
         'max_depth': 6,
         'learning_rate': 0.005,
         "boosting": "gbdt",
         "feature_fraction": 0.4,
         "bagging_freq": 1,
         "bagging_fraction": 0.5,
         "metric": 'mae',
         "lambda_l1": 5,
         "lambda_l2": 1,
         "verbosity": -1}

clf_mae = my_lgb(folds=5, seed=2018)
clf_mae.inference_folds(X_train, y_train, X_test, param)
mae_2018 = clf_mae.oof

In [ ]:
# 0.06398976
clf_mae.submit(output_name = 'model_V1_mae_2018_1')

In [ ]:
param = {'num_leaves': 35,
         'objective':'regression_l1',
         'max_depth': 6,
         'learning_rate': 0.005,
         "boosting": "gbdt",
         "feature_fraction": 0.4,
         "bagging_freq": 1,
         "bagging_fraction": 0.5,
         "metric": 'mae',
         "lambda_l1": 5,
         "lambda_l2": 0,
         "verbosity": -1}

clf_mae_ = my_lgb(folds=5, seed=2019)
clf_mae_.inference_folds(X_train, y_train, X_test, param)
mae_2019 = clf_mae_.oof

In [ ]:
# 0.06399214
clf_mae_.submit(output_name = 'model_V1_mae_2019_1')

param = {'num_leaves': 35,
         'objective':'regression',
         'max_depth': 6,
         'learning_rate': 0.005,
         "boosting": "gbdt",
         "feature_fraction": 0.5,
         "bagging_freq": 1,
         "bagging_fraction": 0.5,
         "metric": 'mse',
         "lambda_l1": 0.16,
         "lambda_l2": 0.1,
         "verbosity": -1}

In [ ]:
p = 0.251
q = 0.25
m = 0.25
n = 0.25
results = mse_2018*p+ mse_2019*q + mae_2018*m + mae_2019*n
print(score(results, y_train))
res = clf.results*p + clf_mse.results*q + clf_mae.results*m + clf_mae_.results*n
submit(model_name='model_V2_bad', predictions=res)

In [ ]:
residual = mae_2018 - y_train
submit(model_name='resdiual_mae', predictions=residual)

In [ ]:
# drop_columns = drop_columns.remove('信用分')
col = train.drop(columns=drop_columns).columns
f = clf_mae.importance_feature(col)
f

In [ ]:
(residual>100).sum()

In [ ]:
feature = ['用户网龄（月）', '当月通话交往圈人数', '用户年龄', 
       '用户账单当月总费用（元）', '用户近6个月平均消费值（元）', '当月金融理财类应用使用总次数', '当月视频播放类应用使用次数',
       '当月网购类应用使用次数', '用户当月账户余额（元）', '近三个月月均商场出现次数',
       '当月旅游资讯类应用使用次数', '缴费用户最近一次缴费金额（元）', '信用分']

In [ ]:
train_df.loc[residual > 100,feature].sample(10)

In [ ]:
train_df.loc[23379, :]